# CSS Lab: Network Structure
This lab demonstrates how to use common tools to analyze the structure of several networks. Topics covered include visualization, centrality measures, shortest paths, and affiliation networks. At the end of the lab, you will be able to visualze and perform an analysis of your own social network.

## Contents

1. [Setup](#Setup)
4. [Community structure](#Community-structure)
    1. [Interpret communities](#Interpret-communities)
5. [Paths](#Paths)
    1. [Find path lengths](#Find-path-lengths)
    2. [Bridges and weak ties](#Bridges-and-weak-ties)
    3. [Find minimum cut](#Find-minimum-cut)
6. [Affiliation networks](#Affiliation-networks)
    1. [Weighted edges](#Weighted-edges)
7. [Your social network](#Your-social-network)
    1. [Visualize your social network](#Visualize-your-social-network)
    2. [Social network communities](#Social-network-communities)
    4. [Find connected component](#Find-connected-component)
    5. [Social network path lengths](#Social-network-path-lengths)
    6. [Find bridges](#Find-bridges)
8. [References](#References)
    


## Setup
We import several libraries, including `networkx` for network algorithms, `pandas` for data processing, and `visJS2jupyter` for visualization. We also define some helper functions used by several of the 

In [ ]:
%pylab inline
import itertools
import json
import math
import urllib.request
import networkx as nx
import networkx as networkx
import networkx.algorithms as nxalg
import networkx.algorithms.community as nxcom
import networkx.readwrite as nxrw
import pandas as pd
import visJS2jupyter.visJS_module as vjs

In [ ]:
# Helper functions

# Standardize a vector to 0 mean and unit variance
def scale(v):
    return (v - v.mean()) / v.std()

# Return a data frame sorted by a node's centrality relative the mean of its centralities
def rel_centrality(df, measure, labels=dict()):
    name = 'rel_{}'.format(measure)
    return pd.DataFrame({
        name: scale(df[measure]) / df['mean_centrality'],
        "label": [labels.get(x, x) for x in df.index]
    })

def path_histogram(G, log=False):
    # Find shortest path length for each pair
    path_lengths = list(itertools.chain(*[
            [length for length in targets.values() if length > 0]
        for source, targets in list(nxalg.shortest_path_length(G))]))
    # Draw histogram with integer bins
    bins = [0.5 + x for x in range(max(path_lengths)+1)]
    counts, centers, patches = plt.hist(
        path_lengths, bins=bins, rwidth=0.8)
    if log:
        plt.yscale('log', nonposy='clip')
        ymax = np.power(10, np.ceil(np.log10(max(counts))))
    else:
        place = np.power(10, np.floor(np.log10(max(counts))))
        ymax = np.ceil(max(counts) / place) * place
    # Plot average shortest path length
    mean_path = nxalg.average_shortest_path_length(G)
    plt.plot([mean_path, mean_path], [1, ymax], label="mean")
    plt.ylim([1, ymax])
    plt.xlabel("Path length")
    plt.ylabel("Count")
    plt.legend()
    plt.tight_layout()

def longest_geodesic(G):
    paths = nxalg.shortest_path(G)
    longest = 0
    longest_path = []
    for source, source_paths in paths.items():
        for target, path in source_paths.items():
            if len(path) > longest:
                longest = len(path)
                longest_path = path
    return path

# Helper functions

def find_bridges(G, labels=None):
    bridges = []
    for v, w in G.edges():
        H = G.copy()
        H.remove_edge(v, w)
        try:
            d = nx.shortest_path_length(H, source=v, target=w)
            if d <= 2:
                continue
        except nx.NetworkXNoPath:
            d = float("Inf")
        if labels is None:
            bridges.append((d, v, w))
        else:
            bridges.append((d, labels[v], labels[w]))
    bridges = sorted(bridges, key=lambda x: x[0], reverse=True)
    for b in bridges:
        if b[0] == float("Inf"):
            print("Global bridge:\t{} <-> {}".format(b[1], b[2]))
        else:
            print("Local bridge, span {}:\t{} <-> {}".format(*b))
            
def find_global_bridges(G, labels=None):
    bridges = nx.algorithms.bridges(G)
    for b in bridges:
        if labels is None:
            v, w = b[0], b[1]
        else:
            v, w = labels[b[0]], labels[b[1]]
        print("Global bridge:\t{} <-> {}".format(v, w))

def get_colors():
    phi = (1 + math.sqrt(5)) / 2
    color = []
    for i in range(1, 20):
        theta = phi * i * math.pi * 2
        x = 128 + math.floor(64*math.sin(theta))
        y = 128 + math.floor(64*math.cos(theta))
        color.append((x, x, y))
    return color

def visualize_visjs(
        G, communities=None, colors=None, default_color="192,192,192",
        node_size_field="node_size", layout="spring", scale=500, pos=None,
        groups=None, weight=None, labels=dict(), edge_width_field="value", title=""):
    # Get list of nodes and edges
    nodes = list(G.nodes())
    edges = list(G.edges())
    # Change node shapes for bipartite graph
    if groups is None:
        node_shapes = dict()
        node_sizes = dict()
        node_colors = dict()
    else:
        node_shapes = dict((n, "square") for n in groups)
        node_sizes = dict((n, 15) for n in groups)
        node_colors = dict((n, "192,128,0") for n in groups)
    # Per-node properties
    nodes_dict = dict((n, {
        "id": labels.get(n, n),
        "node_size": node_sizes.get(n, 5),
        "node_shape": node_shapes.get(n, "dot")
        }) for n in nodes)
    # Generate a layout for the nodes
    edge_smooth_enabled = False
    edge_width = 4
    edge_arrow_scale = 2
    if communities is not None and pos is None:
        # Generate initial positions based on community
        phi = 3.14 / len(nodes)
        community_node = []
        # Create list of nodes and their communities
        for i, com in enumerate(sorted(communities, key=lambda x: len(x), reverse=True)):
            for node in com:
                community_node.append((i, node))
        # Sort by community and
        community_node = sorted(community_node)
        # Generate initial position by placing communities around a circle
        pos = dict((d[1], (math.cos(i*phi), math.sin(i*phi))) for i, d in enumerate(community_node))
    else:
        pos = None
    if layout == "circle":
        pos = nx.circular_layout(G, scale=scale)
    elif layout == "spring":
        pos = nx.spring_layout(G, k=3/math.sqrt(len(nodes)), scale=scale, pos=pos)
    else:
        edge_smooth_enabled = True
    # Assign position
    for n in nodes:
        nodes_dict[n]["x"] = pos[n][0]
        nodes_dict[n]["y"] = pos[n][1]
    # Calculate bounds for scaling
    x_min = min(pos.values(), key=lambda x: x[0])[0]
    x_max = max(pos.values(), key=lambda x: x[0])[0]
    y_min = min(pos.values(), key=lambda x: x[1])[1]
    y_max = max(pos.values(), key=lambda x: x[1])[1]
    x_range = x_max - x_min
    y_range = y_max - y_min
    max_range = max(x_range, y_range)
    # If we have communities, assign color based on community
    if colors is None:
        colors = ["{},{},{}".format(*c) for c in get_colors()]
    if communities is not None:
        for i, com in enumerate(sorted(communities, key=lambda x: len(x), reverse=True)):
            for node in com:
                try:
                    nodes_dict[node]["color"] = "rgba({},1)".format(colors[i])
                    nodes_dict[node]["color_index"] = i
                except IndexError:
                    nodes_dict[node]["color"] = "rgba({},1)".format(default_color)
    # Update color for bipartite nodes
    for node, node_attr in nodes_dict.items():
        if node in node_colors:
            node_attr["color"] = "rgba({},1)".format(node_colors[node])
    # Map node labels to contiguous ids
    node_map = dict(zip(nodes,range(len(nodes))))
    # Determine edge colors
    edge_colors_idx = {}
    for source, target in edges:
        source_color = nodes_dict[source].get("color_index", None)
        target_color = nodes_dict[target].get("color_index", None)
        if source_color == target_color and source_color is not None:
            edge_colors_idx[(source, target)] = source_color
    edge_colors = dict(
        (e,colors[c])
        for e, c in edge_colors_idx.items() if c < len(colors))
    # Per-edge properties, use contiguous ids to identify nodes
    edge_scale = math.ceil(max_range / 200)
    edges_dict = []
    for source, target, data in G.edges(data=True):
        edge = {
            "source": node_map[source],
            "target": node_map[target],
            "title":'test',
            "color": "rgba({},0.3)".format(edge_colors.get((source,target), default_color)),
            "edge_width_field": edge_width_field,
            "value": data.get(edge_width_field, 1) * edge_scale
        }
        if edge_width_field is not None:
            
        edges_dict.append(edge)
    # Convert nodes dict to node list
    nodes_list = [nodes_dict[n] for n in nodes]
    # Check for directed graph
    if G.__class__ == nx.classes.digraph.DiGraph:
        directed = True
    else:
        directed = False
    # Call visjs
    return vjs.visjs_network(
        nodes_list, edges_dict,
        node_size_field="node_size",
        node_size_multiplier=10.0,
        edge_width_field="value",
        edge_width=edge_width,
        edge_arrow_to=directed,
        edge_arrow_to_scale_factor=edge_arrow_scale,
        edge_smooth_enabled=edge_smooth_enabled,
        edge_smooth_type="curvedCW",
        graph_id=hash(title))


## Community structure

What if you didn't know the groups people belonged to and wanted to find them from the network? This is the problem of community detection. There are many ways to find communities. The following cells find and visualize communities using a method that maximizes _modularity_ [CNM2004]. Networks with high modularity have a high number of edges between nodes in the same community and a low number of edges across communities. One benefit of this method is that it determines the best number of communities.

We'll continue using the network from the previous lab, composedof organizations from the American Revolution. In the visualization below, different communities are shown in different colors.

In [ ]:
# Helper functions

def load_revere_affiliation(
        url="https://raw.githubusercontent.com/kjhealy/revere/master/data/PaulRevereAppD.csv"):
    data = urllib.request.urlopen(url)
    df = pd.read_csv(data).set_index("Unnamed: 0")
    people = list(df.index)
    groups = list(df.columns)
    #
    B = nx.Graph()
    for column in df.columns:
        for row in df[df[column] == 1].index:
            B.add_edge(column, row)
    return people, groups, B

def load_revere(
        url="https://raw.githubusercontent.com/kjhealy/revere/master/data/PaulRevereAppD.csv",
        threshold=None, dual=False):
    # Load the affiliation network
    people, groups, B = load_revere_affiliation(url)
    # Project the affiliation network onto the set of people
    if dual:
        G = nxalg.bipartite.projection.weighted_projected_graph(B, groups)
    else:
        G = nxalg.bipartite.projection.weighted_projected_graph(B, people)
    # Threshold
    if threshold is not None:
        for s, t, data in list(G.edges(data=True)):
            if data["weight"] < threshold:
                G.remove_edge(s,t)
    # Copy "weight" data to "value" needed by visjs
    w = nx.get_edge_attributes(G, "weight")
    nx.set_edge_attributes(G, w, "value")
    return G

In [ ]:
people, groups, B = load_revere_affiliation()
G = load_revere()
communities = list(nxcom.greedy_modularity_communities(G))
visualize_visjs(G, communities=communities, scale=1500, title="Community Detection Example")

### Interpret communites
When interpreting the communities, it will be helpful to see lists of the organizations each individual belonged to. The next cell will show that list for each individual. To start interpreting the communities, consider the following questions:
* Do the communities make sense?
* Can you connect particular communities to actual organizations?
* Why would two organizations belong to the same community when two others don't?

In [ ]:
list(nx.neighbors(B, "Church.Benjamin"))

## Affiliation networks
Sometimes it is useful to consider two different types of nodes.
Often, this is the case when the two types of nodes are groups and members of those groups,
with the edges representing membership.
Such networks are called affiliation networks.
The American Revolution data from before was created from an affiliation network.
The cell below visualizes the full affiliation network.
Notice that person nodes are only connected to group nodes and vice versa.

In [ ]:
peopele, groups, B = load_revere_affiliation()
visualize_visjs(B, scale=1500, groups=groups, title="American Revolution Affiliation")

Previously, this affilation network was used to create a network connecting people to people.
But we can just as easily create a network connecting groups to groups (if they have a common member) [Breiger1974]. 

In [ ]:
G = load_revere(dual=True)
visualize_visjs(G, title="American Revolution Groups", edge_width_field=None, groups=G.nodes())

### Weighted edges
In the above example, two groups have an edge between them if they have a common member.
But some groups might have only one member in common, while others have many.
These differences can be represented by adding a _weight_ do the edge.
In this case, weights represent how many members two groups have in common,
but weights can represent anything: distance, similarity, etc.

In [ ]:
visualize_visjs(G, title="American Revolution Weighted", groups=G.nodes(), weight="weight")

## Paths

It is also informative to consider the shortest paths of a network.
For a pair of nodes, the shortest path is the path connecting them in the fewest number of hops.
The example below uses a network of characters from the novel _A Storm of Swords_ [BS2016],
with edges representing characters mentioned at the same time.

In [ ]:
# Helper functions

def load_asoiaf():
    G = nx.Graph()
    with open("external/stormofswords.csv") as f:
        f.readline()
        for row in f:
            source, target, weight = row.split(",")
            G.add_edge(source, target, weight=int(weight), capacity=1)
    return G

First, load and visualize the network.

In [ ]:
G = load_asoiaf()
visualize_visjs(G, title="asoiaf", scale=1000)

### Find path lengths
Next, plot a histogram of the path lengths as well as the mean path length. The diameter of a network is the length of the longest shortest path.
* How do the mean shortest path, mode shortest path, and the longest path compare?
* Do you think these values would stay the same if the network was larger? Why or why not?

In [ ]:
path_histogram(G)

The cell above found the distribution of paths lengths, but it is also informative to look at individual paths. The following cell prints the shortest path between the two most distant nodes in the network. You might expect the endpoints to be very different in some way, perhaps separated geographically, or socially. Similarly the nodes in the middle are ones that connect distant parts of the network, and are likely to play an important role in the network.

In [ ]:
print(longest_geodesic(G))

### Bridges and weak ties
We can also find edges which act as bridges between different parts of the network. An edge is a bridge if removing it disconnects two parts of a network.
More generally, a local bridge connects two nodes that would otherwise be far apart.
If removing an edge causes its endpoints to become distance _n_ apart,
it is called a local bridge of span n.

Bridges are often _weak ties_ [Granovetter1977] because individuals with a strong connection are likely to have common friends.

In [ ]:
find_global_bridges(G)

### Find minimum cut
Sometimes you might want to know where a network is most fragile.
The minimum cut finds the fewest number of links that need to be removed to break a network into two pieces.
In the beginning of the _A Song of Ice and Fire_ series, exiled princess Daenerys Targaryen and her brother Viserys seek to reclaim the throne from reigning king Robert Baratheon.
You can use the minimum cut to find the links that separate characters connected to the Targaryens from those more closely connected to the Baratheons.

First we find the minimum cut: the number of edges separating the two factions.

In [ ]:
mincut, cuts = nxalg.minimum_cut(G, 'Daenerys', 'Robert')
mincut

Now we can visualize the min cut. Who in the "Robert" cluster is closest to members of the "Daenerys" cluster? Why might Daenerys want to identify those people?

In [ ]:
visualize_visjs(G, communities=cuts, scale=1500, title="Mincut")

## Your social network
Now you can do these analyses on your own social network! Specifically, you can download a list of all of your friends and whether they are friends with each other.
This network is called your "ego network."

If you use Facebook, you can use the LostCircles plugin for Chrome to download your social network.
1. Open the following link in Chrome to install LostCircles:
https://chrome.google.com/webstore/detail/lost-circles-facebook-gra/ehpmfdlcppenimpibdifodjgfafkjhjl?hl=en
2. Click the LostCircles icon on the Chrome toolbar and select "Start Loading."
3. Wait until the menu shows "100%" next to "Start Loading."
4. Click the LostCircles icon and then "Download..." followed by "JSON (no pics)". Save the result as `external/LostCircles/egonet.json`.
5. Continue to the next cell

In [ ]:
# Helper functions

def load_lost_circles_json(in_file, name=False):
    with open(in_file) as f:
        raw = json.load(f)
    id_to_name = dict((i, datum["name"]) for i, datum in enumerate(raw['nodes']))
    if name:
        edges = [(id_to_name[datum["source"]], id_to_name[datum["target"]], {"capacity":1}) for datum in raw['links']]
    else:
        edges = [(datum["source"], datum["target"], {"capacity":1}) for datum in raw['links']]
    G = nx.Graph()
    G.add_edges_from(edges)
    return G, id_to_name, edges

### Social network communities
Try finding communities within your social network. Do the communities correspond to the groups you identified above?

In [ ]:
communities = nxalg.community.greedy_modularity_communities(G)
visualize_visjs(G, communities=communities, title="Ego Net Communities", scale=6000, labels=id_to_name)

### Find giant component
Before moving on to examine path length, it is important to consider whether the network is connected. Were there any groups of nodes with no edges going to the rest of the network? If so, some pairs of nodes aren't connected by a path. To get around this, we can examine only the largest connected component, called the _giant component_.

In [ ]:
# Helper function
def giant_component(G):
    giant_component = sorted(nxalg.connected_components(G), reverse=True, key=len)[0]
    for v in set(G.nodes()) - giant_component:
        G.remove_node(v)
    return G

In [ ]:
G = giant_component(G)

### Social network path lengths
Now let's look at the histogram of paths in your social network. How does it compare to the _A Song of Ice and Fire_ network above? Are most paths longer, shorter, or the same? Are any paths much longer?

In [ ]:
path_histogram(G)

Now consider the path between the most distant nodes. Why might the nodes on the ends be so far separated? Why are the nodes in the middle good at connecting them?

In [ ]:
print([id_to_name[i] for i in longest_geodesic(G)])

### Find bridges
Next, consider the bridges in your social network. Do they correspond to weak ties as hypothesized in [Granovetter1977]? Weak ties can be useful for searching a social network because they connect different communities. For example, if you were looking for a job, who would you ask? Do they appear in any bridges?

The cell below will find only global bridges by default. If no global bridges exist, you can change the first line to `local = True` to find local bridges as well. Finding local bridges may take several minutes.

In [ ]:
local = False
if local:
    find_bridges(G, labels=id_to_name)
else:
    find_global_bridges(G, labels=id_to_name)

## Possible extensions

## References

[Healey2013] Healy, K. 2013. "Using Metadata to Find Paul Revere." https://kieranhealy.org/blog/archives/2013/06/09/using-metadata-to-find-paul-revere/

[CNM2004] Clauset, A., Newman, M. E., & Moore, C. (2004). Finding community structure in very large networks. Physical review E, 70(6).

[RLWGBF2017] Rosenthal, S. B., Len, J., Webster, M., Gary, A., Birmingham, A., & Fisch, K. M. (2017). Interactive network visualization in Jupyter notebooks: visJS2jupyter. Bioinformatics.

[BS2016] A. Beveridge and J. Shan, "Network of Thrones," Math Horizons Magazine, Vol. 23, No. 4 (2016), pp. 18-22

[Breiger1974] Breiger, R. L. (1974). The duality of persons and groups. Social forces, 53(2), 181-190.

[Granovetter1977] Granovetter, M. S. (1977). The strength of weak ties. In Social networks (pp. 347-367).